In [1]:
%%capture
%run 3-tfidf-gensim.ipynb

In [2]:
# Fit an LDA model
lda_model = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=20, passes=10)

In [3]:
# Visualize the LDA model using pyLDAvis
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
pyLDAvis.display(vis)